# Classification : this is used to seperate data point into classes of different labels

In [1]:
%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf

# Data Set

In [22]:
Information_column = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', "Species"]
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
#keras(a module inside of Tensorflow) to grab our datasets and read them into a pandas dataframe



train = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv", names= Information_column, header = 0)
test = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv", names= Information_column, header = 0)


In [23]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


# pop the specises column  off and use that as our label

In [24]:
train_y = train.pop('Species')
test_y = test.pop("Species")


# Input Function

In [25]:
def input_fn(features, labels, training = True, num_epochs=10, batch_size = 256):
    # create the input to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels)) 
    if training:
      dataset = dataset.shuffle(1000).repeat()
     
    return dataset.batch(batch_size)

# Feature Columns

In [26]:
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building the Model

In [27]:
#DNNClassifier(Deep Neural Network)
#LinearClassifer

classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpm0_uo64c', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Traning Model

In [ ]:
classifier.train(input_fn = lambda: input_fn(train, train_y, training = True), steps = 5000)

# Evaluation

In [29]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training = False))
print('\nTest set accuracy : {accuracy: 0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-02-24T20:52:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpm0_uo64c/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.24238s
INFO:tensorflow:Finished evaluation at 2022-02-24-20:52:47
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8666667, average_loss = 0.6179079, global_step = 5000, loss = 0.6179079
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpm0_uo64c/model.ckpt-5000

Test set accuracy :  0.867



# Prediction

In [36]:
from tensorflow.python.ops.gen_batch_ops import batch
from tensorflow.core.example.feature_pb2 import Features
def input_fn2(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("please type numeric vaues as prompted.")

for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

    predict[feature]  = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn2(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f})%)'.format(SPECIES[class_id], 100 * probability))





please type numeric vaues as prompted.
SepalLength: 1.4
SepalWidth: 2.5
PetalLength: 2.3
PetalWidth: 4.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpm0_uo64c/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-1.226253  ,  0.35523638,  0.67616   ], dtype=float32), 'probabilities': array([0.07959098, 0.3869866 , 0.53342247], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (53.3)%)
